## Importing libraries

In [1]:
import ifcopenshell
from ifcopenshell import geom
import numpy as np
import open3d as o3d

## Loading IFC file

In [2]:
ifc_file_path = "test_data\PN1833_05_EXE_BIM_000013_01_17IGR_SGO_Structure_ifc2x3.ifc"

ifc_data = ifcopenshell.open(ifc_file_path)
settings = geom.settings()
walls = ifc_data.by_type('IfcWall')

## Extract geometry properties from an IFC entity

In [3]:
def get_geom(settings, entity):
    shape = geom.create_shape(settings, entity)

    ios_vertices = shape.geometry.verts
    ios_edges = shape.geometry.edges
    ios_faces = shape.geometry.faces

    # Grouping vertices by 3 (x,y,z), edges by 2 (edge1, edge2) and faces by 3 (vert1, vert2, vert3)
    vertices = [tuple(ios_vertices[i : i + 3]) for i in range(0, len(ios_vertices), 3)]
    edges = [ios_edges[i : i + 2] for i in range(0, len(ios_edges), 2)]
    faces = [tuple(ios_faces[i : i + 3]) for i in range(0, len(ios_faces), 3)]

    return {'vertices' : vertices, 'edges': edges, 'faces': faces}

## Generate samples on faces and along edges

In [4]:
def sample_edge(v1, v2, num_points=100):
    """Generate points along an edge."""
    return [tuple(v1 + t * (v2 - v1)) for t in np.linspace(0, 1, num_points)]

def sample_face(v1, v2, v3, num_points=1000):
    """Generate points inside a triangular face using barycentric coordinates."""
    points = []
    for _ in range(num_points):
        r1, r2 = np.random.rand(), np.random.rand()
        if r1 + r2 > 1:
            r1, r2 = 1 - r1, 1 - r2
        point = (1 - r1 - r2) * v1 + r1 * v2 + r2 * v3
        points.append(tuple(point))
    return points

## Genrating the point cloud of an entity

In [5]:
def generate_point_cloud(geometry, edge_samples=100, face_samples=1000):
    vertices, edges, faces = geometry["vertices"], geometry["edges"], geometry["faces"]
    point_cloud = set(vertices)  # Start with vertices

    # Sample edges
    for edge in edges:
        v1, v2 = np.array(vertices[edge[0]]), np.array(vertices[edge[1]])
        point_cloud.update(sample_edge(v1, v2, edge_samples))

    # Sample faces
    for face in faces:
        v1, v2, v3 = np.array(vertices[face[0]]), np.array(vertices[face[1]]), np.array(vertices[face[2]])
        point_cloud.update(sample_face(v1, v2, v3, face_samples))

    return list(point_cloud)

## Visualize a point cloud

In [6]:
def visualize_point_cloud_open3d(points):
    """Visualizes a 3D point cloud interactively using Open3D."""
    points = np.array(points)  # Convert list of tuples to NumPy array
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    # Draw the point cloud
    o3d.visualization.draw_geometries([pcd])

## Testing on a random wall

In [7]:
ifc_geometry = get_geom(settings=settings, entity=walls[244])
point_cloud = generate_point_cloud(ifc_geometry, edge_samples=5, face_samples=25)
visualize_point_cloud_open3d(point_cloud)

## Getting global georeference from site

In [16]:
def dms_to_decimal(dms_tuple):
    """Converts (Degrees, Minutes, Seconds, Millionths) to decimal degrees."""
    if not dms_tuple or len(dms_tuple) < 3:
        return 0.0  # Default if data is missing
    degrees, minutes, seconds, millionths = dms_tuple + (0,) * (4 - len(dms_tuple))
    return degrees + minutes / 60 + (seconds + millionths / 1e6) / 3600

def get_global_origin(ifc_file):
    """Extracts the global origin from IfcSite (latitude, longitude, elevation)."""
    site = ifc_file.by_type("IfcSite")
    if site:
        lat = dms_to_decimal(site[0].RefLatitude)
        lon = dms_to_decimal(site[0].RefLongitude)
        elev = site[0].RefElevation if site[0].RefElevation else 0.0

        # print(f"Latitude (Decimal): {lat}")
        # print(f"Longitude (Decimal): {lon}")
        # print(f"Elevation: {elev}")

        return np.array([lon, lat, elev])
    
    return np.array([0, 0, 0])  # Default (0,0,0) if no georeference found

In [15]:
def get_wall_transformation(ifc_entity):
    """Extracts the transformation matrix of an IFC element."""
    if ifc_entity.ObjectPlacement:
        placement = ifc_entity.ObjectPlacement.RelativePlacement
        location = np.array([
            placement.Location.Coordinates[0] if placement.Location else 0,
            placement.Location.Coordinates[1] if placement.Location else 0,
            placement.Location.Coordinates[2] if placement.Location else 0
        ])
        return location
    return np.array([0, 0, 0])  # Default if no placement

def transform_to_global(local_points, global_origin, wall_transform):
    """Transforms local points to global coordinates."""
    return [point + wall_transform + global_origin for point in local_points]

In [ ]:
def process_ifc_walls(ifc_file):
    """Processes all walls in an IFC file and applies georeferencing with orientation."""
    global_origin = get_global_origin(ifc_file)
    walls = ifc_file.by_type("IfcWall")
    
    all_points = []
    
    for wall in walls:
        ifc_geometry = get_geom(settings=settings, entity=wall)
        local_points = generate_point_cloud(ifc_geometry, edge_samples=10, face_samples=100)
        
        # Get wall location and orientation (axis)
        wall_location = get_wall_transformation(wall)
        
        # Transform local points to global points
        global_points = transform_to_global(local_points, global_origin, wall_location)
        
        all_points.extend(global_points)
    
    return all_points